<h1>Cousera Capstone Project<h1>
    Comparing Cities in Midwestern United States

# Importing/Installing Necessary Packages

In [ ]:
import pandas as pd
import numpy as np
!pip install beautifulsoup4
from bs4 import BeautifulSoup

from pandas.io.json import json_normalize
from sklearn.cluster import KMeans

import requests
!pip install wget
import wget
import urllib.request
from urllib.request import urlopen

!pip install geopy
from geopy.geocoders import Nominatim
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib as plt

<p> Reading in Wikipedia Page and Converting to Pandas Dataframe <p>

In [ ]:
path="https://raw.githubusercontent.com/Chrisbutz20/Coursera_Capstone/master/Project_Data_1.csv"
data=pd.read_csv(path,error_bad_lines=False)
data.head()

In [ ]:
data.columns

In [ ]:
data['Population']=data['Population'].str.replace(',', '')
data['Population']=data['Population'].str.replace('.00','')
data['Population'].astype(str).astype(int)
data.head()

In [ ]:
spectrum=({'lg_dec':range(-100,-25)},{'md_dec':range(-25,-15)},{'sm_dec':range(-15,-5)},
          {'no_chg':range(-5,5)},{'sm_inc':range(5,15)},{'md_inc':range(15,25)},{'lg_inc':range(25,100)})

In [ ]:
###Creating a spectrum to plot different color points on map based on growth in population###

In [ ]:
clean_df=data.copy()
print(clean_df)

Create Map of Midwest with Labels for each City <br/>
Format: City (Growth)

In [ ]:
map_ctr='Dubuque,Iowa'
geolocator=Nominatim(user_agent="Foursquare_agent")
location=geolocator.geocode(map_ctr)
t_lat=location.latitude
t_long=location.longitude

In [ ]:
map_mdw=folium.Map(location=[location.latitude,location.longitude],zoom_start=5)

for lat,long,neighbour,growth in zip(clean_df.Latitude,clean_df.Longitude,clean_df.City,clean_df['% Growth (2000 - 2013)']):
    label='{} ({})'.format(neighbour,growth)
    label=folium.Popup(label,parse_html=True)
    folium.features.CircleMarker(
        [lat,long],
        radius=3,
        color='blue',
        popup=label,
        fill=False
    ).add_to(map_mdw)
    
map_mdw

## Four Square API: Neighbourhood Venue Analysis ##

Credentials

In [ ]:
CLIENT_ID = 'J3BXVIECWLAHZXARVRCBB2YM5UHI2AMJURPE1GOOMAJEF4RK' # your Foursquare ID
CLIENT_SECRET = 'JNVENONWCBDNZZ3UG5RTBGJ5TXMCXOPWC0GL3VIQ0AAAG0MC' # your Foursquare Secret
VERSION = '20200806' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

-

New Function "getNearbyVenues" returns venues within the radius of each neighbourhood's Coorindates

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=3000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]["groups"][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
LIMIT=2500
df_venues=getNearbyVenues(names=clean_df.City,
                                   latitudes=clean_df.Latitude,
                                   longitudes=clean_df.Longitude)

In [ ]:
df_venues.head()

In [ ]:
df_venues.shape

In [ ]:
df_venues.groupby('City').count()

Using One-Hot Encoding to format Venue Category for easier analysis

In [ ]:
# one hot encoding
df_onehot = pd.get_dummies(df_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df_onehot['City'] = df_venues['City'] 

# move neighborhood column to the first column
fixed_columns = [df_onehot.columns[-1]] + list(df_onehot.columns[:-1])
df_onehot = df_onehot[fixed_columns]

Calculating frequency of each category for each neighbourhood

In [ ]:
df_grouped=df_onehot.groupby('City').mean().reset_index()

In [ ]:
top_count=10

for city in df_grouped['City']:
    print("----"+city+"----")
    temp = df_grouped[df_grouped['City'] == city].T.reset_index()
    temp.columns = ['Venue','Freq']
    temp = temp.iloc[1:]
    temp['Freq'] = temp['Freq'].astype(float)
    temp = temp.round({'Freq': 2})
    print(temp.sort_values('Freq', ascending=False).reset_index(drop=True).head(top_count))
    print('\n')

New function to return most common venue type for each city.

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Calculating most common venue type for each neighborhood in Toronto

In [ ]:
num_top_venues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['City'] = df_grouped['City']

for ind in np.arange(df_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_grouped.iloc[ind, :], num_top_venues)

venues_sorted.head()

## Begin K-Means Clustering ##

In [ ]:
df_cluster=df_grouped.drop(['City'],1)

Identify # of Clusters (8), perform clustering, and attach cluster labels to new df 

In [ ]:
k_clusters=10
kmeans=KMeans(n_clusters=k_clusters,n_init=20,max_iter=100,random_state=1).fit(df_cluster)

In [ ]:
venues_sorted.insert(0,'Cluster Labels', kmeans.labels_)

In [ ]:
df_merge=clean_df
df_merge=df_merge.join(venues_sorted.set_index('City'), on='City')
df_merge['Cluster Labels']=df_merge['Cluster Labels'].astype(int)
df_merge.head()

In [ ]:
df_merge.head(35)

# Create Map with Cluster Labels 

In [ ]:
# create map
map_clusters = folium.Map(location=[location.latitude, location.longitude], zoom_start=5)

# set color scheme for the clusters
x = np.arange(k_clusters)
ys = [i + x + (i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merge['Latitude'], df_merge['Longitude'], df_merge['City'], df_merge['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Separate Clusters for Easier Analysis

In [ ]:
df_merge.loc[df_merge['Cluster Labels'] == 0]

In [ ]:
df_merge.loc[df_merge['Cluster Labels'] == 1]

In [ ]:
df_merge.loc[df_merge['Cluster Labels'] == 2]

In [ ]:
df_merge.loc[df_merge['Cluster Labels'] == 3]

In [ ]:
df_merge.loc[df_merge['Cluster Labels'] == 4]

In [ ]:
df_merge.loc[df_merge['Cluster Labels'] == 5]

In [ ]:
df_merge.loc[df_merge['Cluster Labels'] == 6]

In [ ]:
df_merge.loc[df_merge['Cluster Labels'] == 7]

In [ ]:
df_merge.loc[df_merge['Cluster Labels'] == 8]

In [ ]:
df_merge.loc[df_merge['Cluster Labels'] == 9]

In [ ]:
df_merge.loc[df_merge['Cluster Labels'] == 10]